In [1]:
import numpy as np
import pandas as pd

from polls import polls

from plot_functions import *

%matplotlib inline

#from bokeh.io import output_notebook
#output_notebook()

In [7]:
YouGovData = pd.read_csv('./VOTER_Survey_December16_Release1.csv')

options = [value[0] for value in polls.values()]
for opt in options:
    print(opt)

Interest in news and public affairs
Political Interest
Direction of the country
Direction of the country (2016)
Trust government to do what's right
Barack Obama Approval
Barack Obama Approval (2016)
Hours watch TV daily
Ideology
Issue Importance: Iraq War
Issue Importance: The Economy
Issue Importance: Immigration
Issue Importance: The Environment
Issue Importance: Terrorism
Issue Importance: Gay Rights
Issue Importance: Education
Issue Importance: Health Care
Issue Importance: Social Security
Issue Importance: The Budget Deficit
Issue Importance: The War in Afganistan
Issue Importance: Taxes
Issue Importance: Medicare
Issue Importance: Abortion
Issue Importance: Crime (2016)
Issue Importance: The Economy (2016)
Issue Importance: Immigration (2016)
Issue Importance: The Environment (2016)
Issue Importance: Religious Liberty (2016)
Issue Importance: Terrorism (2016)
Issue Importance: Gay Rights (2016)
Issue Importance: Family and Medical Leave (2016)
Issue Importance: Health Care (2016)

In [ ]:
#plot_constructor(YouGovData)

In [17]:
bokeh_constructor(YouGovData, polls)

Categorize by: abortidentity_baseline
Plot:presvote16post_2016


In [9]:
keys = [el for el in polls.keys()]
ans_opts = [list(polls[key][3]) for key in keys]
uniques = []
for i in range(len(ans_opts)):
    if not ans_opts[i] in uniques:
        uniques.append(ans_opts[i])

ans_lengths = []
for i in range(len(uniques)):
    if not len(uniques[i]) in ans_lengths:
        ans_lengths.append(len(uniques[i]))
        
poll_keys = [el for el in polls.keys()]
features = []
for i in range(len(YouGovData.columns)):
    if YouGovData.columns[i] in poll_keys:
        features.append(YouGovData.columns[i])
        
features_no_trump = [el for el in features if el != 'fav_trump_2016']
features_no_trump = [el for el in features_no_trump if el != 'Sanders_Trump_2016']
features_no_trump = [el for el in features_no_trump if el != 'presvote16post_2016']
features_no_trump = [el for el in features_no_trump if el != 'presvote08_baseline']
features_no_trump = [el for el in features_no_trump if el != 'pid7_2016']
features_no_trump = [el for el in features_no_trump if el != 'pid7_baseline']
features_no_trump = [el for el in features_no_trump if el != 'pid3_baseline']
features_no_trump = [el for el in features_no_trump if el != 'pid3_2016']
features_no_trump = [el for el in features_no_trump if el != 'immi_muslim_2016']
features_no_trump = [el for el in features_no_trump if el != 'straighttic_baseline']

features_no_trump.append("Target")

features_no_trump = features_no_trump[14:]

In [10]:
fav_trump_or_nah = {
    1 : 1, 
    2 : 1, 
    3 : 2, 
    4 : 2,
    8 : 3,
    np.nan : 4
}

vote_trump_or_nah = {
    2 : 1,
    1 : 2,
    3 : 2,
    4 : 2,
    5 : 2,
    6 : 2,
    7 : 2,
    666: 3
}
#print([(el1, el2) for el1 in YouGovData['case_identifier'] for el2 in YouGovData['presvote16post_2016']])

In [11]:
def encode_target(data, target_column, features, alt_features):
    df_mod = data[features].copy()
    #df_mod = df_mod.fillna(666)
    targets = df_mod[target_column].unique()
    map_to_int = fav_trump_or_nah
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)
    df_mod = df_mod[alt_features].copy()
    
    return (df_mod, targets)

csfy_data, targets = encode_target(YouGovData, 'fav_trump_2016', features, features_no_trump)
print(len(csfy_data["Target"]))

8000


In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer

y = csfy_data["Target"]
x = csfy_data[features_no_trump[:-1]]

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(x)
x = imp.transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1993, test_size=0.08)

print(len(x_train))

dt = DecisionTreeClassifier(random_state=1993)
dt.fit(x_train, y_train)

7360


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1993,
            splitter='best')

In [13]:
predicted = dt.predict(x_test)
accuracy_score(y_test, predicted)

0.8046875

In [ ]:
from sklearn.tree import export_graphviz
import subprocess

In [ ]:
def visualize_tree(tree, feature_names):
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f, feature_names=feature_names)
        
    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        print("Something failed")

visualize_tree(dt, features_no_trump[:-1])

In [ ]:
importances = np.array(dt.feature_importances_)
top_10 = importances.argsort()[-10:][::-1]
print([features_no_trump[i] for i in top_10])
print([importances[i] for i in top_10])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_trees = 1000
dtrf = RandomForestClassifier(n_trees)
dtrf.fit(x_train,y_train)

plt_x = np.linspace(0,len(features)-1,len(features))
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: float("{0:.2f}".format(round(x, 4))), dtrf.feature_importances_), features),
             reverse=True))

#plt.bar(plt_x, dtrf.feature_importances_, width=0.5, color="blue",align='center')
#plt.gca().set_xticklabels(plt_x, rotation=60 )
#plt.xticks(plt_x, features)
#plt.ylabel("relative information")
#plt.xlabel("features")
#plt.show()